In [1]:
# Denpendances
import numpy as np
import pandas as pd
from dLoader import DataLoader

In [2]:
r2000 = pd.read_csv('Russell2000.csv', header=0, index_col=0)

In [3]:
r2000

,Ticker,Company,Sector,Skipped
0,AMC,AMC ENTERTAINMENT HOLDINGS INC CLA,Communication,False
1,OVV,OVINTIV INC,Energy,False
2,NTLA,INTELLIA THERAPEUTICS INC,Health Care,False
3,LSCC,LATTICE SEMICONDUCTOR CORP,Information Technology,False
4,TTEK,TETRA TECH INC,Industrials,False
...,...,...,...,...
2029,GBL,GAMCO INVESTORS INC,Financials,False
2030,PDLI,PDL BIOPHARMA INC,Health Care,True
2031,GTXI,GTXI INC - CVR,Health Care,True
2032,P5N994,Petrocorp Inc Escrow,Financials,True


In [17]:
skip = r2000['Ticker'][r2000['Skipped']].to_list()

In [18]:
import time

while True:    
    wait = False

    db = os.path.abspath('Database')
    for ticker in r2000['Ticker']:
        if os.path.exists(os.path.join(db, ticker + '.csv')) or ticker in skip:
            # print(ticker, 'in Database.')
            continue
        print('Downloading...', end='')
        try:
            DataLoader(ticker)
            if os.path.exists(os.path.join(db, ticker + '.csv')):
                print(ticker, 'Download finish!')
        except Exception as e:
            if 'Unauthorized' in str(e):
                wait = True
                print(e)
                break
            else:
                skip.append(ticker)
                print(e)
    
    if wait:
        time.sleep(60*5)
    else:
        break

In [7]:
# r2000.insert(3, 'Skipped', np.isin(r2000['Ticker'], skip))

In [8]:
# r2000.to_csv('Russell2000.csv')

In [9]:
# Generating gain and loss percentage data
def clip_dataframe(df, p):
    # Batching for test data
    batches = len(df) // p
    max_length = batches * p
    return df.iloc[-max_length:].copy()

def simple_gain_loss(data, period):
    # Get gain and loss values base on start of period and
    # end of period prices
    ndf = clip_dataframe(data, period)
    # Base period prices
    base = np.array(ndf['Close']).reshape(-1, period)[:, 0]
    # Shifting one day to avoid the end period price is the
    # base price price
    shifted = ndf.shift(-1).fillna(method='ffill')
    target = np.array(shifted['Close']).reshape(-1, period)
    # Calculate gain and loss price array
    gain = target.max(1) / base - 1
    loss = target.min(1) / base - 1
    return gain.mean(), loss.mean()

def average_daily_fluctuation(data, minute=30):
    # Calculate an average daily fluctuation percentage value
    hl = data['High'] / data['Low'] - 1
    minutes = 7.5 * (60 / minute)
    return hl.mean() / minutes

def get_values(data, period, minute=30):
    # Getting values of sell limit, stop loss and fluctuation percentage
    sell_limit, stop_loss = simple_gain_loss(data, period)
    fluct = average_daily_fluctuation(data, minute)
    return sell_limit, stop_loss, fluct

In [10]:
# Static back test
def not_so_simple_gain_loss(data, period):
    ndf = clip_dataframe(data, period)
    # Base
    base = np.array(ndf['Close']).reshape(-1, period)[:, 0]
    # Expanding dimension
    base = np.expand_dims(base, 1)
    ndf = clip_dataframe(data, period)
    # base
    base = np.array(ndf['Close']).reshape(-1, period)[:, 0]
    base = np.expand_dims(base, 1)
    # shifted
    shift = ndf.shift(-1).fillna(method='ffill')
    # High
    high = np.array(shift['High']).reshape(-1, period)
    # Low
    low = np.array(shift['Low']).reshape(-1, period)
    # Open
    Open = np.array(shift['Open']).reshape(-1, period)
    # Close
    close = np.array(shift['Close']).reshape(-1, period)
    # Gaining percentage
    high_gains = high / base - 1
    low_gains = low / base - 1
    close_gains = close / base - 1
    open_gains = Open / base - 1
    return open_gains, high_gains, low_gains, close_gains, base.reshape(-1)

def calculate_gains(data, period, sell_limit, stop_loss, fluct):
    open_gains, high_gains, low_gains, close_gains, base = not_so_simple_gain_loss(data, period)
    # Getting gained percentage 
    length = len(open_gains)
    arr = []
    for i in range(length):
        sold = False
        for p in range(period):
            # Main stategy
            if open_gains[i, p] < stop_loss:
                arr.append(open_gains[i, p])
                sold = True
                break
            elif high_gains[i, p] > sell_limit:
                arr.append(high_gains[i, p] - fluct)
                sold = True
                break
            elif low_gains[i, p] < stop_loss:
                arr.append(stop_loss)
                sold = True
                break
        if not sold:
            arr.append(close_gains[i, -1])
    return base, np.array(arr)

def calculate_capital_gain(base, gains, capital=1000, max_share=1000):
    if base[0] > capital:
        capital = base[0] * 10
    original_capital = capital
    length = len(base)
    for i in range(length):
        gain = base[i] * gains[i]
        share = capital // base[i]
        if share > max_share:
            share = max_share
        capital += gain * share
    return capital / original_capital - 1

In [12]:
# Get stock symbol in Database
import os
folder = os.path.abspath('Database')
files = os.listdir(folder)
symbols = [f.split('.')[0] for f in files]
symbols = sorted(symbols)
len(symbols)

2002

In [23]:
for symbol in symbols:
    d1 = DataLoader(symbol).get_data('2018-01-01', '2020-12-31')
    d2 = DataLoader(symbol).get_data('2021-01-01', '2021-12-31')
    print('Checking {} for 2021'.format(symbol))
    print(' Period | Sell Limit % |  Stop Loss % | Capital Gain % ')
    cg = []
    for i in range(3, 21):
        sell_limit, stop_loss, fluct = get_values(d1, i)
        base, gains = calculate_gains(d2, i, sell_limit, stop_loss, fluct)
        capital_gain_percentage = calculate_capital_gain(base, gains)
        print(' {:>6d} | {:>12.2f} | {:>12.2f} | {:>14.2f} '.format(
            i, sell_limit * 100, stop_loss * 100,
            capital_gain_percentage * 100))
        cg.append(capital_gain_percentage)

    print('\nBase on static 3 - 20 days buy sell pattern: ')
    cg = np.array(cg)
    risk = len(cg[cg < 0]) / len(cg)
    leverage = abs(cg.max() / cg.mean())
    print('Risk: {:>6.2f}% | Leverage: {:>6.2f}'.format(
        risk * 100, leverage))
    print('\n')

Checking AAON for 2021
 Period | Sell Limit % |  Stop Loss % | Capital Gain % 
      3 |         1.50 |        -1.22 |          33.70 
      4 |         1.99 |        -1.53 |          17.09 
      5 |         2.63 |        -1.93 |          -2.73 
      6 |         2.76 |        -2.33 |          60.53 
      7 |         3.70 |        -2.07 |          42.16 
      8 |         3.64 |        -2.52 |          18.50 
      9 |         4.42 |        -2.84 |          21.19 
     10 |         4.50 |        -3.12 |         -11.70 
     11 |         4.28 |        -3.78 |          27.34 
     12 |         4.42 |        -4.05 |          11.02 
     13 |         4.99 |        -3.51 |          38.01 
     14 |         6.13 |        -3.14 |          18.79 
     15 |         5.63 |        -4.52 |          -1.30 
     16 |         5.99 |        -3.89 |           3.96 
     17 |         6.62 |        -4.79 |          -1.13 
     18 |         6.67 |        -4.93 |           7.20 
     19 |         6.44 | 

/tmp/ipykernel_17897/4163140549.py:21: RuntimeWarning: Mean of empty slice.
  return gain.mean(), loss.mean()
/home/dx/Documents/code-python/stock-low/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


     19 |          nan |          nan |          44.33 
     20 |          nan |          nan |          45.14 

Base on static 3 - 20 days buy sell pattern: 
Risk:   0.00% | Leverage:   1.06


Checking CHX for 2021
 Period | Sell Limit % |  Stop Loss % | Capital Gain % 
      3 |         2.70 |        -2.36 |          91.66 
      4 |         3.36 |        -3.52 |          86.71 
      5 |         4.88 |        -4.37 |          18.59 
      6 |         5.52 |        -4.50 |          20.05 
      7 |         5.83 |        -5.04 |          -4.61 
      8 |         7.03 |        -6.16 |          45.09 
      9 |         8.28 |        -6.40 |          19.04 
     10 |         9.12 |        -6.59 |          -0.60 
     11 |         9.11 |        -6.56 |           4.18 
     12 |         8.12 |        -8.57 |          13.47 
     13 |         9.13 |        -9.39 |           4.27 
     14 |         9.90 |        -8.59 |         -10.85 
     15 |        10.10 |        -8.91 |         -38.40 


/tmp/ipykernel_17897/921514240.py:61: RuntimeWarning: invalid value encountered in double_scalars
  share = capital // base[i]


     15 |         5.09 |        -5.08 |            nan 
     16 |         4.91 |        -5.33 |            nan 
     17 |         5.93 |        -5.83 |            nan 
     18 |         5.60 |        -5.62 |            nan 
     19 |         4.85 |        -6.36 |            nan 
     20 |         5.75 |        -5.74 |            nan 

Base on static 3 - 20 days buy sell pattern: 
Risk:   0.00% | Leverage:    nan


Checking CVBF for 2021
 Period | Sell Limit % |  Stop Loss % | Capital Gain % 
      3 |         1.15 |        -1.33 |          79.67 
      4 |         1.47 |        -1.70 |          59.05 
      5 |         2.14 |        -1.77 |          17.72 
      6 |         2.17 |        -2.27 |          59.74 
      7 |         2.65 |        -2.36 |          34.20 
      8 |         2.57 |        -2.95 |          51.96 
      9 |         2.92 |        -3.11 |          64.17 
     10 |         3.42 |        -3.17 |          -1.79 
     11 |         3.00 |        -3.34 |          13.36 

In [34]:
'a' in ['ab']

False